In [4]:
%matplotlib notebook

import matplotlib.pyplot as plt
from IPython.display import display
import neurokit2 as nk
import pandas as pd
import numpy as np
import seaborn as sns
import os
import pickle
import xlsxwriter

for zz in range(1,16):

    if zz < 10:
        part_number = '20'+str(zz)
    else:
        part_number = '2'+str(zz)


    # Set matplotlib parameters for displaying graphs
    sns.set_style('darkgrid')
    plt.rcParams['figure.figsize'] = [9, 4.5]  # Bigger images
    plt.rcParams['font.size']= 14
    pd.set_option('display.max_columns', None)

    save_dir = '../data/part'+part_number+'/figures/'

    p = '../data/part'+part_number+'/part'+part_number+'_pilot_ecg_signals.p'
    ecg = pickle.load(open(p,"rb"))
    eda = pickle.load(open('../data/part'+part_number+'/part'+part_number+'_pilot_eda_signals.p',"rb"))
    rsp = pickle.load(open('../data/part'+part_number+'/part'+part_number+'_pilot_rsp_signals.p',"rb"))
    data_file = '../data/part'+part_number+'/part'+part_number+'_pilot.acq'
 
    
    rate = 2000

    data, sampling_rate = nk.read_acqknowledge(data_file)
    data = data.rename(columns={"RSP, X, RSPEC-R": "RSP", "DTU100 - Trigger View, AMI / HLT - A11": "TRIG",
                                "EDA, X, PPGED-R": "EDA", "ECG, X, RSPEC-R": "ECG"})

    timestamps=np.loadtxt('../support/trigger timestamps/trigger timestamps.csv',dtype='int',delimiter=',')
    start_times = timestamps[int(part_number[-2:]) - 1]
    feedback_times = timestamps[int(part_number[-2:]) - 1 + 15]

    taskloads_all = np.loadtxt('../support/taskload settings/taskload settings.csv',dtype='int',delimiter=',',encoding='UTF-8')
    taskload_settings = taskloads_all[int(part_number[-2:]) - 1]

    durations = [100,100,100,100,100,100,100,100,100,100,100,100]
    #durations = [633339, 472624, 462436, 479671, 443789, 460186, 451913, 467455, 465081, 407503, 453908, 453551]

    events = {}
    events['onset'] = start_times
    events['duration'] = durations
    events['label']=  [1,2,3,4,5,6,7,8,9,10,11,12]
    events['condition'] =  taskload_settings
    #events['trust'] = trust_settings DOES NOT WORK!

    events_feedback = {}
    events_feedback['onset'] = feedback_times
    events_feedback['duration'] = durations
    events_feedback['label']=  [1,2,3,4,5,6,7,8,9,10,11,12]
    events_feedback['condition'] =  taskload_settings

    ecg_pilot_interval = nk.epochs_create(ecg, events, sampling_rate=rate, epochs_start=0, epochs_end=50)
    eda_pilot_interval = nk.epochs_create(eda, events, sampling_rate=rate, epochs_start=0, epochs_end=50)
    rsp_pilot_interval = nk.epochs_create(rsp, events, sampling_rate=rate, epochs_start=0, epochs_end=50)


    x_axis_pilot = np.linspace(0, ecg_pilot_interval[1].shape[0]/ sampling_rate, ecg_pilot_interval[1].shape[0])

    hrvs = []
    
    #ecg_pilot_interval starts at index 1
    for index in range(1,13):
        display(nk.ecg_intervalrelated(ecg_pilot_interval[index],sampling_rate=rate))
        hrvs.append(nk.ecg_intervalrelated(ecg_pilot_interval[index], sampling_rate=rate))


    filepath = '../data/part'+part_number+'/Time Based HRV Analyses By Trial.csv'
    f=open(filepath,'ab')
    
    #hrvs starts at index 0
    for iind in range(12):
        np.savetxt(f,hrvs[iind])
    f.close()


,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,HRV_ULF,HRV_VLF,HRV_LF,HRV_HF,HRV_VHF,HRV_LFHF,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,HRV_PAS,HRV_GI,HRV_SI,HRV_AI,HRV_PI,HRV_C1d,HRV_C1a,HRV_SD1d,HRV_SD1a,HRV_C2d,HRV_C2a,HRV_SD2d,HRV_SD2a,HRV_Cd,HRV_Ca,HRV_SDNNd,HRV_SDNNa,HRV_ApEn,HRV_SampEn
0,77.84234,51.430875,771.53125,69.573638,51.838292,0.090176,0.066661,767.25,53.00295,0.069082,76.75,34.375,64.0625,319.5,3.764706,NaN,NaN,NaN,0.02867,0.00353,NaN,NaN,0.890365,-3.551918,36.655208,92.057797,0.398176,10600.982935,2.511452,4.732316,924.794966,0.5625,0.587302,0.837838,0.117647,50.955795,50.966156,50.937415,46.031746,0.508662,0.491338,26.145568,25.696468,0.588997,0.411003,70.650794,59.017821,0.578001,0.421999,53.268778,45.515995,0.471729,1.642228


,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,HRV_ULF,HRV_VLF,HRV_LF,HRV_HF,HRV_VHF,HRV_LFHF,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,HRV_PAS,HRV_GI,HRV_SI,HRV_AI,HRV_PI,HRV_C1d,HRV_C1a,HRV_SD1d,HRV_SD1a,HRV_C2d,HRV_C2a,HRV_SD2d,HRV_SD2a,HRV_Cd,HRV_Ca,HRV_SDNNd,HRV_SDNNa,HRV_ApEn,HRV_SampEn
0,79.653662,43.815259,753.257576,42.680228,44.142126,0.056661,0.058168,748.75,43.7367,0.058413,55.375,25.757576,62.121212,188.5,4.4,NaN,NaN,NaN,0.069459,0.012882,NaN,NaN,0.84355,-2.667024,31.213197,51.700818,0.603727,5069.73822,1.656377,4.411956,342.544163,0.515152,0.538462,0.828571,0.1,48.406375,48.353224,48.460346,52.307692,0.48814,0.51186,21.814694,22.338439,0.417689,0.582311,33.413637,39.452546,0.436517,0.563483,28.216591,32.058612,0.404129,3.218876


,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,HRV_ULF,HRV_VLF,HRV_LF,HRV_HF,HRV_VHF,HRV_LFHF,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,HRV_PAS,HRV_GI,HRV_SI,HRV_AI,HRV_PI,HRV_C1d,HRV_C1a,HRV_SD1d,HRV_SD1a,HRV_C2d,HRV_C2a,HRV_SD2d,HRV_SD2a,HRV_Cd,HRV_Ca,HRV_SDNNd,HRV_SDNNa,HRV_ApEn,HRV_SampEn
0,77.541655,49.895247,775.226562,48.226971,50.283006,0.06221,0.064362,773.5,51.1497,0.066128,63.625,29.6875,64.0625,209.5,3.764706,NaN,NaN,NaN,0.070273,0.012964,NaN,NaN,0.844252,-2.655372,35.555455,58.0254,0.612757,6481.480939,1.631969,4.518644,378.782614,0.609375,0.634921,0.925,0.05,51.430859,51.493018,51.369607,53.968254,0.569173,0.430827,26.831238,23.343724,0.474569,0.525431,39.973111,42.06064,0.500404,0.499596,34.042363,34.0149,0.361108,2.302585


,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,HRV_ULF,HRV_VLF,HRV_LF,HRV_HF,HRV_VHF,HRV_LFHF,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,HRV_PAS,HRV_GI,HRV_SI,HRV_AI,HRV_PI,HRV_C1d,HRV_C1a,HRV_SD1d,HRV_SD1a,HRV_C2d,HRV_C2a,HRV_SD2d,HRV_SD2a,HRV_Cd,HRV_Ca,HRV_SDNNd,HRV_SDNNa,HRV_ApEn,HRV_SampEn
0,79.195187,39.696702,759.823077,46.439092,39.935017,0.061118,0.052245,760.0,44.478,0.058524,62.5,21.538462,61.538462,254.0,4.333333,NaN,NaN,NaN,0.037792,0.008523,NaN,NaN,0.815972,-3.275668,28.238321,57.085845,0.494664,5064.273774,2.021574,4.411487,461.612947,0.507692,0.516129,0.78125,0.133333,53.799318,54.156107,53.477828,53.225806,0.62982,0.37018,22.45264,17.213355,0.501473,0.498527,40.425205,40.306283,0.526781,0.473219,32.697998,30.991096,0.473944,2.047693


,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,HRV_ULF,HRV_VLF,HRV_LF,HRV_HF,HRV_VHF,HRV_LFHF,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,HRV_PAS,HRV_GI,HRV_SI,HRV_AI,HRV_PI,HRV_C1d,HRV_C1a,HRV_SD1d,HRV_SD1a,HRV_C2d,HRV_C2a,HRV_SD2d,HRV_SD2a,HRV_Cd,HRV_Ca,HRV_SDNNd,HRV_SDNNa,HRV_ApEn,HRV_SampEn
0,82.025763,37.881049,733.088235,54.417262,38.119511,0.07423,0.051673,728.75,41.88345,0.057473,57.625,16.176471,60.294118,250.5,3.4,NaN,NaN,NaN,0.037076,0.006168,NaN,NaN,0.857361,-3.294778,26.954565,70.331,0.383253,5955.647601,2.609243,4.481899,734.042588,0.558824,0.575758,0.868421,0.235294,53.201721,53.679964,52.784057,45.454545,0.520453,0.479547,19.469868,18.689093,0.688418,0.311582,58.354299,39.258444,0.66686,0.33314,43.49885,30.74498,0.564241,2.397895


...

,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,HRV_ULF,HRV_VLF,HRV_LF,HRV_HF,HRV_VHF,HRV_LFHF,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,HRV_PAS,HRV_GI,HRV_SI,HRV_AI,HRV_PI,HRV_C1d,HRV_C1a,HRV_SD1d,HRV_SD1a,HRV_C2d,HRV_C2a,HRV_SD2d,HRV_SD2a,HRV_Cd,HRV_Ca,HRV_SDNNd,HRV_SDNNa,HRV_ApEn,HRV_SampEn
0,88.626667,38.257942,677.719178,45.477863,38.516504,0.067104,0.056451,672.0,36.3237,0.054053,51.0,15.068493,60.273973,219.0,4.055556,NaN,NaN,NaN,0.069,0.006999,NaN,NaN,0.907908,-2.673646,27.235281,58.274277,0.467364,4986.072998,2.139661,4.404729,498.748868,0.479452,0.492958,0.742857,0.0,51.4518,51.550814,51.357331,53.521127,0.567402,0.432598,20.520558,17.917849,0.432917,0.567083,38.342377,43.883409,0.457036,0.542964,30.750863,33.517181,0.588372,1.754019


,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,HRV_ULF,HRV_VLF,HRV_LF,HRV_HF,HRV_VHF,HRV_LFHF,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,HRV_PAS,HRV_GI,HRV_SI,HRV_AI,HRV_PI,HRV_C1d,HRV_C1a,HRV_SD1d,HRV_SD1a,HRV_C2d,HRV_C2a,HRV_SD2d,HRV_SD2a,HRV_Cd,HRV_Ca,HRV_SDNNd,HRV_SDNNa,HRV_ApEn,HRV_SampEn
0,89.728227,35.631992,668.294521,48.965254,35.881257,0.073269,0.053318,658.5,36.3237,0.055161,53.5,15.068493,47.945205,234.5,3.65,NaN,NaN,NaN,0.071843,0.00381,NaN,NaN,0.949644,-2.633268,25.37188,64.828532,0.391369,5167.359882,2.555133,4.420239,662.582126,0.493151,0.507042,0.777778,0.0,49.537541,49.508015,49.566261,49.295775,0.509996,0.490004,18.119469,17.760783,0.495397,0.504603,45.629192,46.051225,0.497336,0.502664,34.715546,34.901009,0.632249,1.745239


,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,HRV_ULF,HRV_VLF,HRV_LF,HRV_HF,HRV_VHF,HRV_LFHF,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,HRV_PAS,HRV_GI,HRV_SI,HRV_AI,HRV_PI,HRV_C1d,HRV_C1a,HRV_SD1d,HRV_SD1a,HRV_C2d,HRV_C2a,HRV_SD2d,HRV_SD2a,HRV_Cd,HRV_Ca,HRV_SDNNd,HRV_SDNNa,HRV_ApEn,HRV_SampEn
0,88.928697,33.380852,673.965753,34.634433,33.611014,0.051389,0.049529,673.0,33.3585,0.049567,42.5,15.068493,49.315068,161.5,4.055556,NaN,NaN,NaN,0.152514,0.029302,NaN,NaN,0.838838,-1.880498,23.766576,42.809475,0.555171,3196.36522,1.801247,4.211627,308.44177,0.506849,0.527778,0.842105,0.038462,51.000801,50.967849,51.033071,45.833333,0.486446,0.513554,16.578186,17.033831,0.547355,0.452645,31.671915,28.801753,0.533002,0.466998,25.27792,23.661069,0.473284,2.456736


,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,HRV_ULF,HRV_VLF,HRV_LF,HRV_HF,HRV_VHF,HRV_LFHF,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,HRV_PAS,HRV_GI,HRV_SI,HRV_AI,HRV_PI,HRV_C1d,HRV_C1a,HRV_SD1d,HRV_SD1a,HRV_C2d,HRV_C2a,HRV_SD2d,HRV_SD2a,HRV_Cd,HRV_Ca,HRV_SDNNd,HRV_SDNNa,HRV_ApEn,HRV_SampEn
0,89.222673,32.124,671.684932,40.329761,32.343844,0.060043,0.047826,663.0,39.2889,0.059259,55.5,12.328767,45.205479,200.5,3.842105,NaN,NaN,NaN,0.032375,0.006202,NaN,NaN,0.83922,-3.430375,22.870551,52.364068,0.43676,3762.355937,2.289585,4.28243,479.56791,0.520548,0.541667,0.794872,0.041667,48.773531,48.814063,48.735119,48.611111,0.417124,0.582876,14.773525,17.463846,0.572007,0.427993,39.603533,34.257201,0.547187,0.452813,29.888935,27.189536,0.564374,1.575536


,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_pNN50,HRV_pNN20,HRV_TINN,HRV_HTI,HRV_ULF,HRV_VLF,HRV_LF,HRV_HF,HRV_VHF,HRV_LFHF,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,HRV_PAS,HRV_GI,HRV_SI,HRV_AI,HRV_PI,HRV_C1d,HRV_C1a,HRV_SD1d,HRV_SD1a,HRV_C2d,HRV_C2a,HRV_SD2d,HRV_SD2a,HRV_Cd,HRV_Ca,HRV_SDNNd,HRV_SDNNa,HRV_ApEn,HRV_SampEn
0,86.69917,40.645682,692.711268,47.057007,40.938994,0.067932,0.058676,688.0,43.7367,0.063571,58.0,22.535211,57.746479,210.0,3.227273,NaN,NaN,NaN,0.078336,0.011985,NaN,NaN,0.86731,-2.546749,28.94824,60.442559,0.478938,5496.862628,2.087953,4.447085,504.804831,0.464789,0.470588,0.75,0.0,49.818182,49.814594,49.82233,52.941176,0.503312,0.496688,20.537258,20.401673,0.42471,0.57529,39.390279,45.844398,0.439376,0.560624,31.411567,35.481947,0.428089,1.410987


7
10
14


In [ ]:

for zz in range(1,16):
    if zz == 14:
        continue
    if zz == 7:
        continue
    if zz == 10:
        continue